In [11]:
import sys
sys.path.append('../')
import latexStrings as ls
import numpy as np
import matplotlib.pyplot as plt
from math import *
from IPython.display import Latex
import odesolver
import scipy.linalg as linear

# Ejercicio 2

Tomemos el siguiente PVI: 
\begin{equation} \label{eq:3}
 \begin{array}{c} y_1' = - y_1 + y_2 \\
y_2' =- y_1 - y_2 \end{array} 
\qquad \mbox{con} \qquad \begin{array}{c}
y_1(0)=0 \\ y_2(0)=1 \end{array} \qquad \mbox{para } t\in [0,1] \qquad 
\end{equation}


con solución exacta  $\overrightarrow{y}(t)=\left(\begin{array}{c} y_1(t) \\
y_2(t)\end{array} \right)$ está dada por :
\begin{equation} \label{eq:4}
 \begin{array}{c} y_1(t) = e^{-t}\sin(t) \\
y_2(t) = e^{-t}\cos(t) \end{array}
\end{equation}

Sea $\overrightarrow{W}_j$ el vector correspondiente a la aplicación del método del Punto Medio Implícito con paso $h_j$. La entrada $w_i$ es la aproximación en $t=1$ de $y_i$ para $i,j \in \{1,2\}$

Sea $E_j=\Vert W_j-\overrightarrow{y}(1)\Vert _2$ el error global en $t=1$ para $j \in \{1,2\}$

Aplicaremos el método  para $h_1=0.1$ y $h_2=0.01$:

In [34]:
f = lambda t, y : np.array([-y[0]+y[1], -y[0]-y[1]])
soln = lambda t: np.array([np.exp(-T)*np.sin(T), np.exp(-T)*np.cos(T)])
y0 = np.array([0,1])
T,W = odesolver.solve(f, y0, (0,1), 10, method = 'Implicit Midpoint')
globalError=linear.norm(W[:,10]-soln(T)[:,10])
dif=abs(W[:,10]-soln(T)[:,10]) #error global por entrada en t=1
Latex( ls.latexVector(W[:,10],'W_1',form='%f')+' $E_1 = '+str(globalError)+'$'+ls.latexVector(dif,'G_1',form='%f'))

<IPython.core.display.Latex object>

In [35]:
T,W = odesolver.solve(f, y0, (0,1), 100, method = 'Implicit Midpoint')
globalError=linear.norm(W[:,100]-soln(1)[:,100])
dif=abs(W[:,100]-soln(T)[:,100]) #error global por entrada en t=1
Latex( ls.latexVector(W[:,100],'W_2',form='%f')+'$E_1 = '+str(globalError)+'$'+ls.latexVector(dif,'G_2',form='%f'))

<IPython.core.display.Latex object>